In [2]:
import sys
from pathlib import Path
import shutil
import json
import csv

import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

TMP = DIR/'tmp'
IN_DIR = TMP/'inputs'
OUT_DIR = TMP/'outputs'

%load_ext autoreload
%autoreload 2

# Start fresh
wt.rm_paths(TMP)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# polygons = wt.build_polygons(tids)
# collection = {'type': 'FeatureCollection', 'features': polygons}
# json.dumps(collection)

In [4]:
# # Download SRTM data from NASA

# path = Path('/home/araichev/srtmnz/srtm1')
# url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL3.003/2000.02.11/"
# #url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/"
# tids = wt.SRTM_NZ_TILE_IDS
# for i, tid in enumerate(sorted(tids)):
#     filename = tid + '.SRTMGL3.hgt.zip'
#     href = url + filename
#     print(i, href)

In [5]:
# # Download elevation test data

# TEST_DATA_DIR = DIR/'tests'/'data'
# ts = wt.read_transmitters(TEST_DATA_DIR/'transmitters.csv')[:2]
# lon_lats = wt.get_lon_lats(ts)
# tiles = wt.get_tile_names(lon_lats)
# print('Tiles to download =', tiles)
# wt.download_srtm(tiles, TEST_DATA_DIR/'srtm3', api_key=wt.get_secret('GITLAB_API_KEY'))

In [11]:
high_definition = True

path = DIR/'tests'/'data'/'transmitters.csv'
transmitters = wt.read_transmitters(path)[:2]
for t in transmitters:
    print(t)

{'vertical_beamwidth': '', 'antenna_height': 10.0, 'polarization': 0.0, 'downtilt': '', 'power_eirp': 4.0, 'latitude': -35.712247, 'site_name': '4', 'name': 'GrayWireless_4', 'frequency': 5725.0, 'longitude': 174.336547, 'bearing': '0', 'horizontal_beamwidth': '', 'network_name': 'Gray Wireless'}
{'vertical_beamwidth': '30', 'antenna_height': 20.0, 'polarization': 0.0, 'downtilt': '1', 'power_eirp': 4.0, 'latitude': -35.658841, 'site_name': '5', 'name': 'MyNetwork_5', 'frequency': 5725.0, 'longitude': 174.281534, 'bearing': '0', 'horizontal_beamwidth': '90', 'network_name': 'My Network'}


In [12]:
# Download topography data

lonlats = wt.get_lonlats(transmitters)
tiles = wt.get_tile_ids(lonlats)
print('Tiles to download =', tiles)
if high_definition:
    path = IN_DIR/'srtm1'
else:
    path = IN_DIR/'srtm3'
api_key = wt.get_secret('GITLAB_API_KEY')

wt.download_srtm(tiles, path, high_definition=high_definition, api_key=api_key)

% ls {path}

Tiles to download = {'S36E174'}
S36E174.SRTMGL1.hgt.zip


In [14]:
# Create SPLAT transmitter files

in_path = DIR/'tests'/'data'/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
if out_path.exists():
    shutil.rmtree(str(out_path))
    
wt.create_splat_transmitter_files(in_path, out_path)

% ls {out_path}

CanUCMe_21.az       MyNetwork_11.az   MyNetwork_9.az   Skynet_14.az
CanUCMe_21.el       MyNetwork_11.el   MyNetwork_9.el   Skynet_14.el
CanUCMe_21.lrp      MyNetwork_11.lrp  MyNetwork_9.lrp  Skynet_14.lrp
CanUCMe_21.qth      MyNetwork_11.qth  MyNetwork_9.qth  Skynet_14.qth
CanUCMe_22.az       MyNetwork_5.az    RuralCom_18.az   Skynet_15.az
CanUCMe_22.el       MyNetwork_5.el    RuralCom_18.el   Skynet_15.el
CanUCMe_22.lrp      MyNetwork_5.lrp   RuralCom_18.lrp  Skynet_15.lrp
CanUCMe_22.qth      MyNetwork_5.qth   RuralCom_18.qth  Skynet_15.qth
CanUCMe_23.az       MyNetwork_6.az    RuralCom_19.az   Skynet_16.az
CanUCMe_23.el       MyNetwork_6.el    RuralCom_19.el   Skynet_16.el
CanUCMe_23.lrp      MyNetwork_6.lrp   RuralCom_19.lrp  Skynet_16.lrp
CanUCMe_23.qth      MyNetwork_6.qth   RuralCom_19.qth  Skynet_16.qth
GrayWireless_4.az   MyNetwork_7.az    Skynet_12.az     Skynet_17.az
GrayWireless_4.el   MyNetwork_7.el    Skynet_12.el     Skynet_17.el
GrayWireless_4.lrp  MyNetwork_7.lrp   Skyn

In [15]:
# Create SPLAT topography files

if high_definition:
    in_path = IN_DIR/'srtm1'
else:
    in_path = IN_DIR/'srtm3'
out_path = OUT_DIR/'splat_files'

wt.create_splat_topography_files(in_path, out_path, high_definition=high_definition)

% ls {out_path}

-36:-35:185:186-hd.sdf  MyNetwork_11.az   MyNetwork_9.el   Skynet_14.lrp
CanUCMe_21.az           MyNetwork_11.el   MyNetwork_9.lrp  Skynet_14.qth
CanUCMe_21.el           MyNetwork_11.lrp  MyNetwork_9.qth  Skynet_15.az
CanUCMe_21.lrp          MyNetwork_11.qth  RuralCom_18.az   Skynet_15.el
CanUCMe_21.qth          MyNetwork_5.az    RuralCom_18.el   Skynet_15.lrp
CanUCMe_22.az           MyNetwork_5.el    RuralCom_18.lrp  Skynet_15.qth
CanUCMe_22.el           MyNetwork_5.lrp   RuralCom_18.qth  Skynet_16.az
CanUCMe_22.lrp          MyNetwork_5.qth   RuralCom_19.az   Skynet_16.el
CanUCMe_22.qth          MyNetwork_6.az    RuralCom_19.el   Skynet_16.lrp
CanUCMe_23.az           MyNetwork_6.el    RuralCom_19.lrp  Skynet_16.qth
CanUCMe_23.el           MyNetwork_6.lrp   RuralCom_19.qth  Skynet_17.az
CanUCMe_23.lrp          MyNetwork_6.qth   Skynet_12.az     Skynet_17.el
CanUCMe_23.qth          MyNetwork_7.az    Skynet_12.el     Skynet_17.lrp
GrayWireless_4.az       MyNetwork_7.el    Skynet_12.lrp  

In [ ]:
# Create coverage reports

transmitter_names = [t['name'] for t in transmitters[:1]]
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitter_names, 
  high_definition=high_definition)

% ls {out_path}

Timing create_coverage_reports...
2016-08-17 12:47:51.072356   Began process


In [ ]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path)

% ls {path}